# Presentación del notebook

**Feature Engineering para Predicción Semanal de Bollería**

Este notebook tiene como objetivo preparar el dataset final que será utilizado para entrenar un modelo de predicción de ventas semanales en la categoría **Bollería**. Parte de los datos diarios ya validados y realiza todo el proceso de agregación, enriquecimiento, generación de variables temporales y preparación de los conjuntos de datos `X` e `y`.



🗂️ Entradas
- `validated_range_diario.parquet`: archivo diario con los datos corregidos, sin valores nulos, y con columnas de contexto como `is_weekend`, `is_easter`, `is_summer_peak`, etc.



⚙️ Proceso realizado

1. **Filtrado por familia "Bollería"** y selección de variables clave (`fecha`, `base_imponible`, `total`, etc.).
2. **Agregación semanal** usando el lunes como primer día de la semana.
3. **Asignación precisa de la variable `is_easter`** usando fechas exactas de Semana Santa (no solo la ISO week).
4. **Cálculo de variables contextuales** a nivel semanal como:
   - `is_summer_peak`: si hay algún día de verano "pico" en la semana.
   - `is_easter`: si hay algún día de Semana Santa en la semana.
5. **Generación de variables lag** (`lag_1`, `lag_2`, `lag_3`, `lag_4`, `lag_12`, `lag_24`, `lag_52`) sobre la variable `base_imponible` para capturar patrones temporales semanales y anuales.
6. **Eliminación de filas con valores nulos** introducidos por los lags.
7. **Selección de variables de entrada (X) y target (y)** para el entrenamiento del modelo:
   - **Features**: lags + variables contextuales (`is_summer_peak`, `is_easter`)
   - **Target**: `base_imponible`
8. **Exportación final del dataset** a `ts_df_bolleria_baseline.parquet` listo para el entrenamiento del modelo.



📤 Salidas
- `ts_df_bolleria_semanal.parquet`: serie semanal enriquecida (previa a aplicar lags).
- `ts_df_bolleria_baseline.parquet`: dataset final con lags y sin nulos, listo para modelar.



📌 Notas
- La lógica de `is_easter` se basa en las fechas exactas de Semana Santa, no en semanas ISO, para reflejar mejor el comportamiento real del negocio.
- Los lags se calculan sólo hacia el pasado (nunca con información futura).
- Este notebook no entrena modelos, solo **deja todo listo para el siguiente paso: entrenamiento**.





In [1]:
import pandas as pd
from dateutil.easter import easter
import os

# 1. Carga del diario corregido
df_diario = pd.read_parquet('../data/interim/validated_range_fecha_familia_20250630.parquet')
df_diario['fecha'] = pd.to_datetime(df_diario['fecha'])

# 2. Recalcular is_easter en el diario por si no estuviera:
iso = df_diario['fecha'].dt.isocalendar()
df_diario['year_iso'] = iso['year']
df_diario['week_iso'] = iso['week']

df_diario['is_easter'] = df_diario.apply(
    lambda r: int(r['week_iso'] == easter(r['year_iso']).isocalendar()[1]),
    axis=1
)

# 3. Agregar a nivel semanal (todas las familias)
df_semanal = (
    df_diario
      .groupby(['year_iso','week_iso','familia'], as_index=False)
      .agg({
         'base_imponible': 'sum',
         'is_summer_peak': 'max',
         'is_easter':      'max'
      })
)

# 4. Filtrar solo BOLLERIA
df_bolleria_semanal = df_semanal.query("familia=='BOLLERIA'").copy()

# 5. Renombrar columnas y ordenar
df_bolleria_semanal.rename(columns={
    'year_iso':'year',
    'week_iso':'week'
}, inplace=True)
df_bolleria_semanal = df_bolleria_semanal.sort_values(['year','week']).reset_index(drop=True)

# 6. Guardar en processed
processed_dir = '../data/processed'
os.makedirs(processed_dir, exist_ok=True)
out_path = os.path.join(processed_dir, 'ts_df_bolleria_semanal.parquet')
df_bolleria_semanal.to_parquet(out_path, index=False)

print(f"Nuevo ts_df_bolleria_semanal guardado en: {out_path}")
print("Comprobación is_easter por año:", df_bolleria_semanal.groupby('year')['is_easter'].sum().to_dict())


Nuevo ts_df_bolleria_semanal guardado en: ../data/processed\ts_df_bolleria_semanal.parquet
Comprobación is_easter por año: {np.uint32(2023): 1, np.uint32(2024): 1, np.uint32(2025): 1}


In [3]:
import pandas as pd
import os

# 1. Carga de la serie semanal procesada
df = pd.read_parquet('../data/processed/ts_df_bolleria_semanal.parquet')

# 2. Asegurar orden temporal
df = df.sort_values(['year','week']).reset_index(drop=True)

# 3. Definición de los lags a crear
selected_lags = [1, 2, 3, 4, 12, 24, 52]

# 4. Generar las columnas lag_X
for lag in selected_lags:
    df[f'lag_{lag}'] = df['base_imponible'].shift(lag)

# 5. Eliminar filas con NaN (introducidas por los lags)
df_features = df.dropna().reset_index(drop=True)

# 6. Selección de las columnas para el baseline
feature_cols = [f'lag_{lag}' for lag in selected_lags] + ['is_summer_peak','is_easter']
target_col   = 'base_imponible'

X = df_features[feature_cols]
y = df_features[target_col]

print("Shape X:", X.shape)
print("Features ejemplo:")
print(X.head())

# 7. (Opcional) Guardar este dataset listo para modelado
out_dir = '../data/processed'
os.makedirs(out_dir, exist_ok=True)
df_features.to_parquet(os.path.join(out_dir,'ts_df_bolleria_baseline.parquet'), index=False)
print("Dataset de features guardado en:", os.path.join(out_dir,'ts_bolleria_baseline.parquet'))


Shape X: (79, 9)
Features ejemplo:
    lag_1   lag_2   lag_3   lag_4      lag_12   lag_24  lag_52  \
0  789.71  534.79  563.18  806.54  912.130000  1745.97  825.11   
1  571.14  789.71  534.79  563.18  750.990000  1681.41  658.40   
2  572.51  571.14  789.71  534.79  821.840000  1753.02  741.40   
3  597.65  572.51  571.14  789.71  895.391469  1835.18  653.64   
4  680.30  597.65  572.51  571.14  743.259393  2127.71  680.46   

   is_summer_peak  is_easter  
0               0          0  
1               0          0  
2               0          0  
3               0          0  
4               0          0  
Dataset de features guardado en: ../data/processed\ts_bolleria_baseline.parquet
